In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
print('PyTorch version:[%s].'%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device:[%s].'%(device))

PyTorch version:[1.12.1+cu113].
device:[cuda:0].
